# Control Systems
## Introduction
In this tutorial, I will attempt to provide you with a solid understanding of how varying levels of control systems work. Explanations will be accompanied by example programs aimed to set a precedent for how these systems can be implemented in the Autonomous Vehicle (AV) Challenge. Topics mentioned later in this notebook will build upon topics mentioned before, so it is important to read, run, and understand the code provided before moving on to the next section.

## Background
Before we move on to the main topic (control systems), I just want to mention briefly some prerequisites for getting the most of this code. While the philosophy of control systems is independent of the specific feedback (sensors) you are using when building your car, and I am not fully aware of what the specifications of the car you have assembled are, I will make use of some common sensors well-suited to this application. Those sensors and some boilerplate code for how to operate them will be provided below. ALso, as a disclaimer, I know that you are using the BBC micro:bit processor so the code I am writting pertaining to sensor control is written in MicroPython and may will not run in this notebook. And regardless we won't be able to interact with any actual sensors in here so I created some 'mock' functions where I just stub out fake feedback.

### Ultrasonic Distance Sensors
Without getting too much into detail, an ultrasonic sensor uses high-frequency sound waves to calculate distance by measuring the time it takes for the sound to bounce off nearby objects

In [2]:
import time

# Controls the pulses emitted
trigger_pin = 0
# Measures the pulses received
echo_pin = 1

def calculateDistance():
  # Clear trigger by setting it to LOW
  trigger_pin.digital_write(0)
  # Set trigger to HIGH
  trigger_pin.write_digital(1)

  # Set trigger to LOW after 0.01ms
  time.sleep_us(10)
  trigger_pin.write_digital(0)

  start_time = time.ticks_us()
  stop_time = time.ticks_us()

  # Save the time the pulse is emitted
  # The hardware will automatically set echo_pin to HIGH once the pulse is sent
  # so once we detect HIGH we start the timer
  while echo_pin.read_digital() == 0:
      start_time = time.ticks_us()

  # Save the time the echo is detected
  # When echo_pin detects a signal it will go LOW
  while echo_pin.read_digital() == 1:
      stop_time = time.ticks_us()

  # Time difference between start and arrival
  elapsed_time = time.ticks_diff(stop_time, start_time)

  # Multiply with the speed of sound (34300 cm/s)
  # and divide by 2, since the sound wave has to travel to and from the object
  distance = (elapsed_time * 34300) / (2 * 1000000)

  # Distance in cm
  return distance

def calculateDistanceMock(distance):
  return distance

## Photoresistors
Photoresistors are electronic components that change their resistance based on the amount of light that they are exposed to. By measuring the resistance of a photoresistor, you can determine the amount of light that is currently hitting it.


In [ ]:
left_photo_pin = 2
right_photo_pin = 3

def calculatePhotoresistance():
    return (left_photo_pin.read_analog(), right_photo_pin.read_analog())

## Rotary Encoder
A rotary encoder is a device that converts the rotation of a shaft into a digital signal. It typically consists of a shaft with a rotating wheel, which has slots or markings on it that pass by a sensor, generating pulses that can be counted to determine the distance traveled. In most cases, the markings on the wheel are magnets and the sensor they pass by is a Hall magnetic effect sensor (which will detect the presence of a magnetic field).

Since we will know the circumference of the wheel, we will know how much distance is traveled per rotation.

Ex: A wheel with a diameter of 5cm will have a circumference of roughly 31cm. So if our wheel has 1 magnet, each rotation detected will be equivalent to 31cm traveled.

In [ ]:
import math

# Set up the hall magnetic effect sensor pin
hall_pin = 4
hall_pin.set_pull(hall_pin.PULL_UP)

# Set up the rotary encoder
diameter = 5 # cm
circumference = diameter * math.pi
count_per_rotation = 5 # 5 magnets evenly spaced along the wheel
distance_per_count = circumference / count_per_rotation

# Set up the initial state
previous_state = 0
current_state = 0
distance = 0

def recordDistance():
    # Read the current state of the hall magnetic effect sensor
    current_state = hall_pin.read_digital()
    
    # If the state has changed, increment the distance variable
    if current_state != previous_state:
        distance += distance_per_count
    
    # Update the previous state
    previous_state = current_state
    
    # Pause briefly to avoid reading the sensor too quickly
    time.sleep_ms(10)

## Servo
To reiterate, I am not entirely sure what your car setup for this year is but in years prior we have modified the RC cars used in races so that is what I will be doing here. With that, we know that the car is steered by a single 180 degree servo motor. Here is some code to control that servo

In [ ]:
servo_pin = 5

def steer(angle):
  servo_pin.write_analog(angle * 1024 // 180)
  time.sleep(1)

## With that out of the way
Let's start learning about how we can use feedback from these sensors to power our control systems.

## Open-Loop Control
Let's start off with the most straightforward example of a control system. In an open-loop system, we can forget about the sensors we just learned about for a second since we will not be receiving any feedback from them.

Instead, open-loop control would be like coding a predetermined route for the car to traverse, setting it in motion, then letting it traverse the route without input from sensors or a controller. In the past, there was an aspect of the competition requiring us to measure a route by hand, then code our car to traverse it on the fly. However, looking at the instructions for this years competition it doesn't seem to be a factor. Be that as it may, it is still an important to understand the how it works and also to understand when and how it fails.

In [ ]:
motor_pin = 6

def open_loop_control():
    # Drive forward for 3 seconds
    motor_pin.write_analog(1023)
    time.sleep(3)
    motor_pin.write_analog(0)

    # Turn wheels to the right
    servo_pin.write_analog((120 / 180) * 1023)

    # Drive forward with wheels turned
    motor_pin.write_analog(1023)
    time.sleep(2)

    # Stop
    motor_pin.write_analog(0)